# Data Exploration

## Setup

### Downloading Librabies

In [2]:
%pip install pandas
%pip install spacy
%pip install nltk
%pip install scikit-learn
%pip install tensorflow
%pip install user-agents

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
^C
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### Importing Librabies

In [25]:
import pandas as pd
from user_agents import parse
import numpy as np

## Reading Data

### Read User behaviors and Non statistical exploration

#### Typing Data

In [4]:
key_stroke = pd.read_csv("../DataSets/UserBehaviors/Keystroke/free-text.csv")

key_stroke = key_stroke.iloc[:, :-1]

# Get the unique user IDs
#first_10_user_ids = key_stroke['participant'].unique()[:10]

# Filter the data for the first 10 unique users
#first_10_users_data = key_stroke[key_stroke['participant'].isin(first_10_user_ids)]

# Drop unneeded columns
#first_10_users_data = first_10_users_data.drop(first_10_users_data.columns[-1], axis=1)

# Display head
key_stroke.isnull().sum()

/var/folders/jw/dxpsx41s5xlf3dsmc4b4f4tc0000gn/T/ipykernel_4407/53650565.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  key_stroke = pd.read_csv("../DataSets/UserBehaviors/Keystroke/free-text.csv")


participant       0
session           0
key1              0
key2            198
DU.key1.key1      0
DD.key1.key2    198
DU.key1.key2    198
UD.key1.key2    198
UU.key1.key2    198
dtype: int64

#### Mouse movement && Session Info

In [5]:
session_info = pd.read_csv("../DataSets/UserBehaviors/mousedynamics/EVTRACKINFO/EVTRACKINFO.csv", sep='\t')
mouse_movements = pd.read_csv("../DataSets/UserBehaviors/mousedynamics/EVTRACKTRACK/EVTRACKTRACK.csv", sep='\t')


# Filter uneed data
mouse_data = mouse_movements[mouse_movements['event'].str.contains('mouse', case=False, na=False)]

# Display head
# Drop unneeded columns
del session_info['_id']
del mouse_data['_id']
del mouse_data['cursor']
del session_info['documentw']
del session_info['documenth']
del session_info['date']
del mouse_data['key']

print(mouse_data.isnull().sum())
print(session_info.isnull().sum())

timestamp     0
xpos          0
ypos          0
event         0
user          0
session_id    0
dtype: int64
user          0
session_id    0
ua            0
screen        0
screenh       0
windoww       0
windowh       0
dtype: int64


In [8]:
def extract_browser_info_and_os(ua_string):
    # Parse the user agent string
    user_agent = parse(ua_string)

    # Extract browser and operating system
    browser = user_agent.browser
    os = user_agent.os
    device =  user_agent.device

    return {"browser": str(browser), "os": str(os), "device": str(device)}

# Apply extraction to session_info
session_info[['browser', 'os', 'device']] = session_info['ua'].apply(lambda ua: pd.Series(extract_browser_info_and_os(ua)))

session_info

,user,session_id,ua,screen,screenh,windoww,windowh,browser,os,device
0,Alluserspreauth,366758616,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,1920,1080,1920,951,"Browser(family='Chrome', version=(76, 0, 3809)...","OperatingSystem(family='Linux', version=(), ve...","Device(family='Other', brand=None, model=None)"
1,User1,366758624,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,1920,1080,1920,951,"Browser(family='Chrome', version=(76, 0, 3809)...","OperatingSystem(family='Linux', version=(), ve...","Device(family='Other', brand=None, model=None)"
2,Alluserspreauth,366758629,Mozilla/5.0 (X11; Linux x86_64; rv:73.0) Gecko...,1920,1080,1856,919,"Browser(family='Firefox', version=(73, 0), ver...","OperatingSystem(family='Linux', version=(), ve...","Device(family='Other', brand=None, model=None)"
3,User2,366758657,Mozilla/5.0 (X11; Linux x86_64; rv:73.0) Gecko...,1920,1080,1856,919,"Browser(family='Firefox', version=(73, 0), ver...","OperatingSystem(family='Linux', version=(), ve...","Device(family='Other', brand=None, model=None)"
4,Alluserspreauth,366758800,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,1920,1080,1853,900,"Browser(family='Chrome', version=(80, 0, 3987)...","OperatingSystem(family='Linux', version=(), ve...","Device(family='Other', brand=None, model=None)"
...,...,...,...,...,...,...,...,...,...,...
342,Alluserspreauth,366993675,Mozilla/5.0 (Linux; Android 9; Mi A1) AppleWeb...,360,640,360,263,"Browser(family='Chrome Mobile', version=(80, 0...","OperatingSystem(family='Android', version=(9,)...","Device(family='XiaoMi Mi A1', brand='XiaoMi', ..."
343,User1,366993687,Mozilla/5.0 (Linux; Android 9; Mi A1) AppleWeb...,360,640,360,560,"Browser(family='Chrome Mobile', version=(80, 0...","OperatingSystem(family='Android', version=(9,)...","Device(family='XiaoMi Mi A1', brand='XiaoMi', ..."
344,User6,366999237,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,1920,1080,1850,981,"Browser(family='Chrome', version=(79, 0, 3945)...","OperatingSystem(family='Linux', version=(), ve...","Device(family='Other', brand=None, model=None)"
345,User11,367012614,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:73....,1920,1080,1920,951,"Browser(family='Firefox', version=(73, 0), ver...","OperatingSystem(family='Ubuntu', version=(), v...","Device(family='Other', brand=None, model=None)"


## LSTM Model

In [9]:
# Merge mouse_data with session_info based on user and session_id
merged_data = mouse_data.merge(session_info[['user', 'session_id', 'browser', 'os']], on=['user', 'session_id'], how='left')

print("\nMerged Data with Browser and OS:")
print(len(merged_data))
print(len(mouse_data))

# Count unique operating systems
unique_os_count = merged_data['os'].nunique()
print(f"Number of unique operating systems: {unique_os_count}")

# Count unique browsers
unique_browsers_count = merged_data['browser'].nunique()
print(f"Number of unique browsers: {unique_browsers_count}")


Merged Data with Browser and OS:
29220
29220
Number of unique operating systems: 3
Number of unique browsers: 5


In [27]:
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

le_merchant = LabelEncoder()
merged_data['browser'] = le_merchant.fit_transform(merged_data['browser'].values)

le_transaction_type = LabelEncoder()
merged_data['os'] = le_transaction_type.fit_transform(merged_data['os'].values)

# Count unique operating systems
unique_os_count = merged_data['os'].nunique()
print(f"Number of unique operating systems: {unique_os_count}")

# Count unique browsers
unique_browsers_count = merged_data['browser'].nunique()
print(f"Number of unique browsers: {unique_browsers_count}")

print(len(merged_data[merged_data["user"] == "Alluserspreauth"]))
print(len(merged_data[merged_data["user"] != "Alluserspreauth"]))
print(len(merged_data))

print(merged_data)

model_mouse_data = merged_data[merged_data["user"] != "Alluserspreauth"]

Number of unique operating systems: 3
Number of unique browsers: 5
1136
28084
29220
                     timestamp    xpos   ypos      event             user  \
0      2020-02-17 08:35:44.485   343.0   65.0  mousemove  Alluserspreauth   
1      2020-02-17 08:35:44.785  1010.0  436.0  mousemove  Alluserspreauth   
2      2020-02-17 08:35:44.985   894.0  628.0   mouseout  Alluserspreauth   
3      2020-02-17 08:35:45.187   884.0  645.0  mousemove  Alluserspreauth   
4      2020-02-17 08:35:45.392   879.0  645.0  mousemove  Alluserspreauth   
...                        ...     ...    ...        ...              ...   
29215  2020-02-25 09:05:42.182   503.0  287.0   mouseout            User1   
29216  2020-02-25 09:05:42.382   446.0  172.0   mouseout            User1   
29217  2020-02-25 09:05:42.582   440.0  156.0  mousemove            User1   
29218  2020-02-25 09:05:46.228   440.0  156.0  mousemove            User1   
29219  2020-02-25 09:05:46.432   254.0   46.0   mouseout            U

In [38]:
key_stroke.rename(columns={'participant': 'user'}, inplace=True)
# Step 2: Limit to the first 11 users
selected_users = key_stroke['user'].drop_duplicates().head(11)

x = 1
# Step 3: Rename users to Userx format
for user in selected_users:
    key_stroke.loc[key_stroke['user'] == user, 'user'] = "User" + str(x)
    x = x+1

key_stroke = key_stroke[key_stroke["user"].str.contains("User", case=False, na=False)]

# Step 2: Create a pseudo-timestamp for keystrokes
key_stroke['timestamp'] = pd.date_range(start='2020-02-17 09:35:49.089692', periods=60129, freq='1s')

key_stroke

ValueError: Length of values (60129) does not match length of index (60107)

In [12]:
# Step 1: Merge mouse and keystroke data
# We will use a time window to align keystrokes with mouse events
time_window = 5  # Define a time window of 5 seconds

def merge_with_time_window(mouse_data, keystroke_data, time_window):
    # Ensure timestamps are in datetime format
    mouse_data['timestamp'] = pd.to_datetime(mouse_data['timestamp'])
    keystroke_data['timestamp'] = pd.to_datetime(keystroke_data['timestamp'])
    
    # Ensure time_window is an integer or float
    if not isinstance(time_window, (int, float)):
        raise ValueError("time_window must be an int or float representing seconds.")
    
    merged_data = []
    for _, mouse_row in mouse_data.iterrows():
        # Get the user ID and timestamp for the mouse movement
        user_id = mouse_row['user']
        mouse_time = mouse_row['timestamp']
        
        # Find keystrokes within the time window
        keystrokes_within_window = keystroke_data[
            (keystroke_data['user'] == user_id) & 
            (keystroke_data['timestamp'] >= mouse_time - pd.Timedelta(seconds=time_window)) & 
            (keystroke_data['timestamp'] <= mouse_time + pd.Timedelta(seconds=time_window))
        ]
        
        for _, keystroke_row in keystrokes_within_window.iterrows():
            # Collect relevant information for each mouse and keystroke pair
            merged_data.append({
                'mouse_timestamp': mouse_time,
                'user': user_id,
                'xpos': mouse_row['xpos'],
                'ypos': mouse_row['ypos'],
                'keystroke_timestamp': keystroke_row['timestamp'],
                'key1': keystroke_row['key1'],
                'key2': keystroke_row['key2'],
                'DU.key1.key1': keystroke_row['DU.key1.key1'],
                'DD.key1.key2': keystroke_row['DD.key1.key2'],
                'DU.key1.key2': keystroke_row['DU.key1.key2'],
                'UD.key1.key2': keystroke_row['UD.key1.key2'],
                'UU.key1.key2': keystroke_row['UU.key1.key2'],
                'event': mouse_row['event'],
                'os': mouse_row['os'],
                'browser': mouse_row['browser']
            })

    # Convert the merged data into a DataFrame
    merged_df = pd.DataFrame(merged_data)
    
    return merged_df

In [66]:
# Merge mouse and keystroke data
combined_data = merge_with_time_window(model_mouse_data, key_stroke, time_window)
print(combined_data)

combined_data.dropna(inplace=True)

# Step 2: Feature Engineering
combined_data['key_encoded_1'] = LabelEncoder().fit_transform(combined_data['key1'].fillna(''))  # Encode keys
combined_data['key_encoded_2'] = LabelEncoder().fit_transform(combined_data['key2'].fillna(''))  # Encode keys
combined_data['event'] = LabelEncoder().fit_transform(combined_data['event'].fillna(''))  # Encode keys
combined_data['browser'] = LabelEncoder().fit_transform(combined_data['browser'].fillna(''))  # Encode keys
combined_data['os'] = LabelEncoder().fit_transform(combined_data['os'].fillna(''))  # Encode keys

standard_scaler = StandardScaler()
combined_data[['xpos', 'ypos', 'DU.key1.key1', 'DD.key1.key2', 'DU.key1.key2', 'UD.key1.key2', 'UU.key1.key2']] = standard_scaler.fit_transform(combined_data[['xpos', 'ypos', 'DU.key1.key1', 'DD.key1.key2', 'DU.key1.key2', 'UD.key1.key2', 'UU.key1.key2']])


# Step 3: Prepare data for LSTM
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        label = data[i + seq_length]  # This would be the next step to predict
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

# Select features for LSTM
features = combined_data[['xpos', 'ypos', 'key_encoded_1', 'key_encoded_2', 'event', 'os', 'browser', 'DU.key1.key1', 'DD.key1.key2', 'DU.key1.key2', 'UD.key1.key2', 'UU.key1.key2']].values
SEQ_LENGTH = 12
X, y = create_sequences(features, SEQ_LENGTH)

# Verify the shapes of the output
print("Shape of X (sequences):", X.shape)
print("Shape of y (labels):", y.shape)

/var/folders/jw/dxpsx41s5xlf3dsmc4b4f4tc0000gn/T/ipykernel_4407/2620238902.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mouse_data['timestamp'] = pd.to_datetime(mouse_data['timestamp'])


             mouse_timestamp   user    xpos   ypos        keystroke_timestamp  \
0    2020-02-17 09:36:02.384  User1   349.0  519.0 2020-02-17 09:35:58.089692   
1    2020-02-17 09:36:02.384  User1   349.0  519.0 2020-02-17 09:35:59.089692   
2    2020-02-17 09:36:02.384  User1   349.0  519.0 2020-02-17 09:36:00.089692   
3    2020-02-17 09:36:02.384  User1   349.0  519.0 2020-02-17 09:36:01.089692   
4    2020-02-17 09:36:02.384  User1   349.0  519.0 2020-02-17 09:36:02.089692   
...                      ...    ...     ...    ...                        ...   
6354 2020-02-17 15:42:16.246  User5  1152.0  110.0 2020-02-17 15:42:17.089692   
6355 2020-02-17 15:42:16.246  User5  1152.0  110.0 2020-02-17 15:42:18.089692   
6356 2020-02-17 15:42:16.246  User5  1152.0  110.0 2020-02-17 15:42:19.089692   
6357 2020-02-17 15:42:16.246  User5  1152.0  110.0 2020-02-17 15:42:20.089692   
6358 2020-02-17 15:42:16.246  User5  1152.0  110.0 2020-02-17 15:42:21.089692   

       key1   key2 DU.key1.

In [67]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, RepeatVector, TimeDistributed, Dense, Dropout
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

# Data Preprocessing (Check NaNs, shape adjustments)

# Step 4: Train-Test Split
#X_train, X_test, y_train, y_test = train_test_split(X, X, test_size=0.2, random_state=42)  # y_train = X_train for reconstruction

# Build the LSTM Model
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(SEQ_LENGTH, X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))  # Dropout to prevent overfitting
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(RepeatVector(SEQ_LENGTH))
model.add(LSTM(32, activation='relu', return_sequences=True))
model.add(Dropout(0.2))  # Another dropout
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(X.shape[2])))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Step 6: Train the Model with Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(X, X, epochs=10, batch_size=8, validation_split=0.2, callbacks=[early_stopping])

# Step 7: Evaluate the Model
#predictions = model.predict(X)
#mse = np.mean(np.square(predictions - y_test), axis=1)

# Define a threshold for anomalies
#threshold = np.percentile(mse, 95)
#anomalies = mse > threshold
#print("Number of Anomalies Detected:", np.sum(anomalies))

Epoch 1/10


/Users/batsiraidzimati/Library/Python/3.9/lib/python/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


635/635 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 30.4294 - val_loss: 9.9882
Epoch 2/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 15.0502 - val_loss: 8.4000
Epoch 3/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 12.9539 - val_loss: 7.8716
Epoch 4/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 12.0783 - val_loss: 7.6003
Epoch 5/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 11.0590 - val_loss: 6.8483
Epoch 6/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 10.0804 - val_loss: 6.3274
Epoch 7/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 9.3001 - val_loss: 6.2872
Epoch 8/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 8.5875 - val_loss: 5.5679
Epoch 9/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 8.2269 - val_loss: 5.7795
Epoch 10/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 7.5744 - val_loss: 5.0285


In [75]:
from keras.models import load_model

# Load user-specific data
# Assuming you have a function to prepare user-specific data
user = combined_data[combined_data['user']=='User1']
features = user[['xpos', 'ypos', 'key_encoded_1', 'key_encoded_2', 'event', 'os', 'browser', 'DU.key1.key1', 'DD.key1.key2', 'DU.key1.key2', 'UD.key1.key2', 'UU.key1.key2']].values
X_user, y_user = create_sequences(features, SEQ_LENGTH)  # User-specific dataset

# Load the pre-trained model
#pretrained_model = load_model('pretrained_lstm_model.h5')

# Fine-tune the model on user-specific data
# Optionally, you can freeze some layers if the dataset is very small
for layer in model.layers[:-2]:  # Freeze all layers except the last two
    layer.trainable = False

# Compile the model again after modifying the layers
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Fine-tune the model
history_user = model.fit(X_user, X_user, epochs=5, batch_size=8, validation_split=0.2)

# Save the fine-tuned model
#pretrained_model.save('fine_tuned_model_user.h5')

Epoch 1/5
107/107 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 6.8976 - val_loss: 8.3298
Epoch 2/5
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 6.5565 - val_loss: 8.5548
Epoch 3/5
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 6.3530 - val_loss: 8.6037
Epoch 4/5
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 6.3165 - val_loss: 9.5555
Epoch 5/5
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 6.1509 - val_loss: 9.6063
